# Timeline

In [ ]:
# from IPython.core.display import HTML
# HTML('''<div class="flourish-embed flourish-cards" data-src="visualisation/1786965" data-url="https://flo.uri.sh/visualisation/1786965/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>''')

# Declaration

* According to MoHFW https://www.mohfw.gov.in/ some new cases have been reassigned states as per latest information
* Confirmed cases includes both Indian and Foriegn citizen
* Cured cases includes cured, discharged and migrated patients 

# Library

In [ ]:
# # utility libraries
# import json
# from datetime import timedelta
# from urllib.request import urlopen

# # converter
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# # hide warnings
# # import warnings
# # warnings.filterwarnings('ignore')

In [ ]:
# storing and anaysis
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# geographical ploting
# import folium
# import geopandas as gpd

In [ ]:
# offline plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

# Data

### 1. API Data

In [ ]:
! ls ../input/covid19-corona-virus-india-dataset

In [ ]:
# MoHFW data
# ==========

# read data
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv')

# convert datetime datatype
df['Date'] = pd.to_datetime(df['Date'])

# rename columns
df.columns = ['Date', 'State', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Recovered', 'New cases', 'New deaths', 'New recovered']

# active cases
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

# first few rows
df.head()

In [ ]:
# daily
# =====

# read data
daily = pd.read_csv('../input/covid19-corona-virus-india-dataset/nation_level_daily.csv')

# convert datetime datatype
daily['Date'] = pd.to_datetime(daily['Date'] + ' 2020')

# get no. of active cases
daily['Total Active'] = daily['Total Confirmed'] - daily['Total Deceased'] - daily['Total Recovered']

# Deaths / 100 cases
daily['Deaths / 100 Cases'] = round((daily['Total Deceased'] / daily['Total Confirmed'])*100, 2)
# Recovered / 100 cases
daily['Recovered / 100 Cases'] = round((daily['Total Recovered'] / daily['Total Confirmed'])*100, 2)
# Deaths / 100 recovered
daily['Deaths / 100 Recovered'] = round((daily['Total Deceased'] / daily['Total Recovered'])*100, 2)

# first few rows
daily.head()

In [ ]:
# global latest
# =============

# read data
world = pd.read_csv('../input/corona-virus-report/country_wise_latest.csv')

# create a new column wether the country is 'India'
world['Is India?'] = world['Country/Region']=='India'

# first few rows
world.head()

In [ ]:
# states_latest
# =============

# read data
states_latest = pd.read_csv('../input/covid19-corona-virus-india-dataset/state_level_latest.csv')

# remove the row containing 'Total'
states_latest = states_latest[states_latest['State']!='Total']

# mortality rate
states_latest['Mortality Rate'] = round((states_latest['Deaths'] / states_latest['Confirmed'])*100, 2)

# recovery rate
states_latest['Recovery Rate'] = round((states_latest['Recovered'] / states_latest['Confirmed'])*100, 2)

# first few rows
states_latest.head()

In [ ]:
# district level latest
# =====================

# read data
dist_latest = pd.read_csv('../input/covid19-corona-virus-india-dataset/district_level_latest.csv')

# remove the column containing unassigned state values
dist_latest = dist_latest[dist_latest['State'] != 'State Unassigned']

# mortality rate
dist_latest['Mortality Rate'] = round((dist_latest['Deceased'] / dist_latest['Confirmed'])*100, 2)
# recovery rate
dist_latest['Recovery Rate'] = round((dist_latest['Recovered'] / dist_latest['Confirmed'])*100, 2)

# first few rows
dist_latest.head()

In [ ]:
# tests day by day
# ================

# read data
tests_dbd = pd.read_csv('../input/covid19-corona-virus-india-dataset/tests_day_wise.csv')

# to datetime datatype
tests_dbd['Update Time Stamp'] = tests_dbd['Update Time Stamp'].str.extract('(\d{1,2}/\d{1,2}/\d{4})')
tests_dbd['Update Time Stamp'] = pd.to_datetime(tests_dbd['Update Time Stamp'], format='%d/%m/%Y')

# first few rwos
tests_dbd.head()

In [ ]:
# state wise tests statistics
# ===========================

# read data
state_tests = pd.read_csv('../input/covid19-corona-virus-india-dataset/tests_state_wise.csv')

# to datetime datatype
# tests_dbd['Update Time Stamp'] = pd.to_datetime(tests_dbd['Update Time Stamp'])

# first few rwos
state_tests.head()

In [ ]:
# Patient level data
# ==================

# importing dataset
p_df = pd.read_csv('../input/covid19-corona-virus-india-dataset/patients_data.csv')
# p_df.head(5)

# fixing date format
for col in ['Date Announced', 'Estimated Onset Date', 'Status Change Date']:
    p_df[col] = pd.to_datetime(p_df[col], format='%d/%m/%Y')

# fix nationality values
p_df['Nationality'] = p_df['Nationality'].replace('Indian', 'India')

# first few rows
p_df.head(3)

### 2. Map data

In [ ]:
# # District level shape file
# # =========================

# # district level shape file
# dist_shape = gpd.read_file('../input/india-district-wise-shape-files/output.shp')
# # subset columns
# dist_shape = dist_shape[['objectid', 'statecode', 'statename', 'state_ut', 
#                          'distcode', 'distname', 'geometry']]
# # rename states
# dist_shape['statename'] = dist_shape['statename'].str.replace('&', 'and')
# dist_shape['statename'] = dist_shape['statename'].str.replace('NCT of ', '')
# dist_shape['statename'] = dist_shape['statename'].str.replace('Chhatisgarh', 'Chhattisgarh')
# dist_shape['statename'] = dist_shape['statename'].str.replace('Orissa', 'Odisha')
# dist_shape['statename'] = dist_shape['statename'].str.replace('Pondicherry', 'Puducherry')
# # dist_shape.head()


# # State level shape file
# # ======================

# # groupby state to get state level shape file
# states_shape = dist_shape.dissolve(by='statename').reset_index()
# # states_shape.head()

In [ ]:
# # grouped district file
# # =====================

# dist_count = p_df.groupby(['detected_district'])['patient_number'].count().reset_index()
# dist_count.columns = ['district', 'count']
# # dist_count.head()


# # grouped state file
# # ==================

# state_count = p_df.groupby(['detected_state'])['patient_number'].count().reset_index()
# state_count.columns = ['state', 'count']
# # state_count.head()

In [ ]:
# # District map
# # ============

# # merge shape file with count file
# dist_map = pd.merge(dist_shape, dist_count, left_on='distname', right_on='district', how='left')
# # droping missing values
# dist_map = dist_map[~dist_map['count'].isna()]
# # fixing datatype
# dist_map['count'] = dist_map['count'].astype('int')
# # dist_map.head(3)


# # States map
# # ==========

# # merge shape file with count file
# state_map = pd.merge(states_shape, state_count, left_on='statename', right_on='state', how='right')
# # fill na with 0
# state_map['count'] = state_map['count'].fillna(0)
# # state_map.head(3)

# Latest Data

In [ ]:
# Latest Data
# ===========

# sort based on confirmed cases
latest = states_latest.sort_values('Confirmed', ascending=False).reset_index(drop=True)

# remove 'State Unassigned' row
latest = latest[latest['State']!='State Unassigned']

# rearrange columns
latest = latest.loc[:, ['State', 'Confirmed', 'Active', 'Deaths', 'Mortality Rate', 
                        'Recovered', 'Recovery Rate']]

# background color
latest.style\
    .background_gradient(cmap="Blues", subset=['Confirmed', 'Active'])\
    .background_gradient(cmap="Greens", subset=['Deaths', 'Mortality Rate'])\
    .background_gradient(cmap="Reds", subset=['Recovered', 'Recovery Rate'])

In [ ]:
# subset columns
temp = latest[['Active', 'Deaths', 'Recovered']]

# rename columns
temp.columns = ['Active', 'Deaths', 'Cured']

# melt into longer format
tm = temp.melt(value_vars=['Active', 'Deaths', 'Cured'])

# plot
fig_1 = px.treemap(tm, path=["variable"], values="value", height=250, 
                 color_discrete_sequence=[act, rec, dth], title='Latest stats')
fig_1.data[0].textinfo = 'label+text+value'
fig_1.show()

# Over Time

In [ ]:
def plot_daily(col, hue):
    fig = px.bar(daily, x="Date", y=col, title=col, 
                 color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
plot_daily('Total Confirmed', '#000000')

In [ ]:
plot_daily('Daily Confirmed', '#000000')

In [ ]:
plot_daily('Total Active', '#000000')

In [ ]:
plot_daily('Total Deceased', dth)

In [ ]:
plot_daily('Daily Deceased', dth)

In [ ]:
plot_daily('Total Recovered', rec)

In [ ]:
plot_daily('Daily Recovered', rec)

In [ ]:
def plot_daily(col, hue):
    temp = tests_dbd.copy()
    # temp = temp[~temp[col].isna()]
    fig = px.scatter(temp, x="Update Time Stamp", 
                 y=col, title=col, 
                 color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
plot_daily('Total Samples Tested', dth)

In [ ]:
# stacked bar chart

# subset columns
temp = daily[['Date', 'Total Active', 'Total Deceased', 'Total Recovered']]

# melt data
temp = temp.melt(value_vars=['Total Recovered', 'Total Deceased', 'Total Active'],
                 id_vars="Date", var_name='Case', value_name='Count')
temp.head()

# plot
fig_2 = px.bar(temp, x="Date", y="Count", color='Case', 
               title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig_2.show()

In [ ]:
# stacked bar chart

# subset columns
temp = daily[['Date', 'Daily Confirmed', 'Daily Deceased', 'Daily Recovered']]

# melt data
temp = temp.melt(value_vars=['Daily Recovered', 'Daily Deceased', 'Daily Confirmed'],
                 id_vars="Date", var_name='Case', value_name='Count')
temp.head()

# plot
fig_2 = px.bar(temp, x="Date", y="Count", color='Case', 
               title='Daily cases over time', color_discrete_sequence = [rec, dth, act])
fig_2.show()

In [ ]:
def plot_stacked(col):
    temp = df.sort_values(col, ascending=False)
    fig = px.bar(df, x="Date", y=col, color='State', 
                 color_discrete_sequence = px.colors.qualitative.Vivid)
    fig.update_layout(title='State wise '+ col, xaxis_title="", yaxis_title="", 
                      showlegend=False)
    fig.show()

In [ ]:
plot_stacked('Confirmed')

In [ ]:
plot_stacked('Deaths')

In [ ]:
plot_stacked('Recovered')

In [ ]:
plot_stacked('New cases')

In [ ]:
plot_stacked('New deaths')

In [ ]:
plot_stacked('New recovered')

In [ ]:
# Daily statistics
# ================

temp = daily[daily['Total Confirmed'] > 100]

fig_c = px.line(temp, x="Date", y="Deaths / 100 Cases", color_discrete_sequence=['#000000'])
fig_d = px.line(temp, x="Date", y="Recovered / 100 Cases", color_discrete_sequence=['#649d66'])
fig_r = px.line(temp, x="Date", y="Deaths / 100 Recovered", color_discrete_sequence=['#ff677d'])

fig = make_subplots(rows=1, cols=3, shared_xaxes=False, 
                    subplot_titles=('No. of Deaths to 100 Cases', 
                                    'No. of Recovered to 100 Cases', 
                                    'No. of Deaths to 100 Recovered'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)
fig.add_trace(fig_r['data'][0], row=1, col=3)

In [ ]:
temp = pd.DataFrame(df.groupby('Date').apply(len), columns=['No. of states affected']).reset_index()
temp['Total no. of States/UT'] = 36
temp = temp.melt(id_vars='Date', value_vars=['No. of states affected', 'Total no. of States/UT'])
temp.head()

fig_s = px.line(temp, x="Date", y="value", color='variable', line_dash='variable', 
                color_discrete_sequence=['#000000', '#ef1223'])
fig_s.update_layout(title='No. of States/UT affected', 
                  xaxis_title='', yaxis_title='')
fig_s.show()

In [ ]:
temp = daily.loc[:, ['Date', 'Total Active', 'Total Recovered']]
temp = temp.melt(id_vars='Date', value_vars=['Total Active', 'Total Recovered'])
temp.head()

fig_c = px.line(temp, x="Date", y="value", color='variable', line_dash='variable', 
                color_discrete_sequence=[dth, rec])
fig_c.update_layout(title='Active vs Recovered cases', 
                  xaxis_title='', yaxis_title='')
fig_c.show()

# Where does India Stands

In [ ]:
def plot_topn(col, n):
    df = world.sort_values(col, ascending=False).head(n)
    fig = px.bar(df, x=col, y='Country/Region', color='Is India?', text=col, 
                 orientation='h', color_discrete_sequence=['#393e46', '#d92027'])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="", 
                      yaxis_categoryorder = 'total ascending',
                      showlegend=False)
    fig.show()

In [ ]:
plot_topn('Confirmed', 10)

In [ ]:
plot_topn('1 week change', 10) 

In [ ]:
plot_topn('Deaths', 15)

In [ ]:
plot_topn('Recovered', 10)

In [ ]:
wm = pd.read_csv('../input/corona-virus-report/worldometer_data.csv')
wm['Is India?'] = wm['Country/Region']=='India'

In [ ]:
px.scatter(wm, x='TotalCases', y='TotalDeaths', color='Is India?', hover_data=['Country/Region'],
           log_x=True, log_y=True, title='Confirmed vs Deaths',
           color_discrete_sequence=['#393e46', '#d92027'])

In [ ]:
px.scatter(wm, x='Tot Cases/1M pop', y='Tests/1M pop', color='Is India?', 
           log_x=True, log_y=True, hover_data=['Country/Region'], title='Tot Cases/1M pop vs Tests/1M pop',
           color_discrete_sequence=['#393e46', '#d92027'])

# Leading States

In [ ]:
def plot_topn_states(col, color, n):
    df = states_latest.sort_values(col, ascending=False).head(n)
    fig = px.bar(df, x=col, y='State', text=col, 
                 orientation='h', 
                 color_discrete_sequence=[color])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="", 
                      yaxis_categoryorder = 'total ascending',
                      showlegend=False)
    fig.show()

In [ ]:
plot_topn_states('Confirmed', '#333333', 10)

In [ ]:
plot_topn_states('Active', '#333333', 10)

In [ ]:
plot_topn_states('Deaths', dth, 10)

In [ ]:
plot_topn_states('Mortality Rate', dth, 10)

In [ ]:
plot_topn_states('Recovered', rec, 10)

In [ ]:
plot_topn_states('Recovery Rate', rec, 10)

In [ ]:
px.scatter(states_latest[states_latest['Confirmed']>10000], x='Confirmed', y='Deaths', 
           color='State', size='Confirmed', text='State', 
           log_x =True, log_y=True, title='Confirmed vs Death (Only States with more thatn 10000 cases)')

In [ ]:
# fig = px.scatter(df.sort_values('State', ascending=False), 
#                  x='Date', y='State', size='New cases', color='New cases', 
#                  color_continuous_scale=px.colors.sequential.Aggrnyl)
# fig.update_layout(yaxis = dict(dtick = 1))
# fig.update_layout(title='No. of new cases in each state', xaxis_title="", yaxis_title="")
# fig.update_layout(height=900)
# fig.update_layout(coloraxis_showscale=False)
# fig.show()

In [ ]:
fig = px.scatter(df.sort_values('State', ascending=False), 
                 x='Date', y='State', size='Active', color='Active', 
                 color_continuous_scale=px.colors.sequential.thermal)
fig.update_layout(yaxis = dict(dtick = 1))
fig.update_layout(title='No. of active cases in each state', xaxis_title="", yaxis_title="")
fig.update_layout(height=900)
fig.update_layout(coloraxis_showscale=False)
fig.show()

In [ ]:
plt.style.use('seaborn')
g = sns.FacetGrid(df.sort_values(['State', 'Date']), 
                  col="State", hue="State", 
                  sharey=False, col_wrap=7)
g = g.map(plt.plot, "Date", "Active")
g.set_xticklabels(rotation=90)
g.fig.subplots_adjust(top=.9)
g.fig.suptitle('Active case in each state over time', 
               fontsize=20)
plt.show()

# District level

In [ ]:
def plot_topn_dist(x, th, n):
    
    df = dist_latest[dist_latest['Confirmed']>th]
    df = df[df['District']!='Unknown']
    df = df[df['District']!='Other State']
    df = df.sort_values(x, ascending=False).head(n)
    
    title = str(x) +'(Only districts with more than ' + str(th) +' cases)'
    
    fig = px.bar(df, x=x, y='District', color='State', 
                 text=x, orientation='h', 
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.update_layout(title=title, xaxis_title="", yaxis_title="", 
                      yaxis_categoryorder = 'total ascending')
    fig.show()

In [ ]:
plot_topn_dist('Confirmed', 100, 10)

In [ ]:
plot_topn_dist('Active', 100, 10)

In [ ]:
plot_topn_dist('Deceased', 100, 10)

In [ ]:
plot_topn_dist('Mortality Rate', 100, 10)

In [ ]:
plot_topn_dist('Recovered', 100, 10)

In [ ]:
plot_topn_dist('Recovery Rate', 100, 10)

In [ ]:
temp = dist_latest.groupby(['State', 'District'])['Confirmed'].sum().reset_index()
temp.head()
fig = px.treemap(temp, path=["State", "District"], values="Confirmed", 
                 height=700, title='Number of Confirmed Cases', 
                 color_discrete_sequence = px.colors.qualitative.Vivid)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
# leg_kwds={'title':'No. of case',
#           'loc': 'upper right',
#           'ncol':1}

# fig, ax = plt.subplots(figsize=(10, 10))
# dist_shape.plot(ax=ax, color='whitesmoke')
# dist_map.plot(column='count', cmap='GnBu', ax=ax, legend=True, 
#               k=5, scheme='Quantiles', legend_kwds=leg_kwds)
# plt.title('District level')
# ax.set_axis_off()
# plt.show()

# Patient wise Data

In [ ]:
def print_missing(col):
    print('Total no. of values :', p_df.shape[0], \
          '\nNo. of missing values :', p_df.shape[0]-p_df[[col]].dropna().shape[0], \
          '\nNo. of available values :', p_df.shape[0]-(p_df.shape[0]-p_df[[col]].dropna().shape[0]))

In [ ]:
print_missing('Age Bracket')

px.histogram(p_df, x='Age Bracket', color_discrete_sequence = ['#35495e'], 
             nbins=50, title='Distribution of ages of confirmed patients')

In [ ]:
def print_missing_vals():
    print('Total no. of values :', p_df.shape[0], 
          '\nNo. of missing values :', p_df.shape[0]-temp.shape[0], 
          '\nNo. of available values :', p_df.shape[0]-(p_df.shape[0]-temp.shape[0]))

In [ ]:
fig = make_subplots(
    rows=1, cols=2, column_widths=[0.8, 0.2],
    subplot_titles = ['Gender vs Age', ''],
    specs=[[{"type": "histogram"}, {"type": "pie"}]]
)

temp = p_df[['Age Bracket', 'Gender']].dropna()

print_missing_vals()

gen_grp = temp.groupby('Gender').count()

fig.add_trace(go.Histogram(x=temp[temp['Gender']=='F']['Age Bracket'], nbinsx=50, name='Female', marker_color='#6a0572'), 1, 1)
fig.add_trace(go.Histogram(x=temp[temp['Gender']=='M']['Age Bracket'], nbinsx=50, name='Male', marker_color='#39065a'), 1, 1)

fig.add_trace(go.Pie(values=gen_grp.values.reshape(-1).tolist(), labels=['Female', 'Male'], marker_colors = ['#6a0572', '#39065a']),1, 2)

fig.update_layout(showlegend=False)
fig.update_layout(barmode='stack')
fig.data[2].textinfo = 'label+text+value+percent'

fig.show()

In [ ]:
fig = make_subplots(
    rows=1, cols=2, column_widths=[0.8, 0.2],
    subplot_titles = ['Cases vs Age', ''],
    specs=[[{"type": "histogram"}, {"type": "pie"}]]
)

temp = p_df[['Age Bracket', 'Current Status']].dropna()

print_missing_vals()

gen_grp = temp.groupby('Current Status').count()

fig.add_trace(go.Pie(values=gen_grp.values.reshape(-1).tolist(), labels=['Deceased', 'Hospitalized', 'Recovered'], 
                     marker_colors = ['#fd0054', '#393e46', '#40a798'], hole=.3),1, 2)

fig.add_trace(go.Histogram(x=temp[temp['Current Status']=='Deceased']['Age Bracket'], nbinsx=50, name='Deceased', marker_color='#fd0054'), 1, 1)
fig.add_trace(go.Histogram(x=temp[temp['Current Status']=='Recovered']['Age Bracket'], nbinsx=50, name='Recovered', marker_color='#40a798'), 1, 1)
fig.add_trace(go.Histogram(x=temp[temp['Current Status']=='Hospitalized']['Age Bracket'], nbinsx=50, name='Hospitalized', marker_color='#393e46'), 1, 1)

fig.update_layout(showlegend=False)
fig.update_layout(barmode='stack')
fig.data[0].textinfo = 'label+text+value+percent'

fig.show()

In [ ]:
p_df['Type of transmission'] = p_df['Type of transmission'].replace('Imported ', 'Imported')
p_df['Type of transmission'] = p_df['Type of transmission'].replace('Unknown', 'TBD')

temp = pd.DataFrame(p_df[['Type of transmission']].groupby('Type of transmission')['Type of transmission'].count())
temp = temp.dropna()
temp.columns = ['count']
temp = temp.reset_index().sort_values(by='count')

print('Total no. of values :', p_df.shape[0], '\nNo. of missing values :', p_df.shape[0]-temp['count'].sum(), '\nNo. of available values :', temp['count'].sum())

fig = px.bar(temp, x='count', y='Type of transmission', orientation='h', text='count', width=600, height=300,
       color_discrete_sequence = ['#35495e'], title='Type of transmission')
fig.update_xaxes(title='')
fig.update_yaxes(title='')
fig.show()

# Week wise

In [ ]:
daily['Month'] = pd.DatetimeIndex(daily['Date']).month
daily['Week No.'] = daily['Date'].dt.strftime('%U')
daily.head()

In [ ]:
week_wise = daily.groupby('Week No.')['Daily Confirmed', 'Total Confirmed', 'Daily Recovered', 
                                      'Total Recovered', 'Daily Deceased', 'Total Deceased'].sum().reset_index()
week_wise.columns = ['Week No.', 'New Confirmed', 'Total Confirmed', 'New Recovered', 
                      'Total Recovered', 'New Deceased', 'Total Deceased']
week_wise.head()

In [ ]:
def plot_weekly(col, hue):
    fig = px.bar(week_wise, x="Week No.", y=col, title=col, 
                 width=700, color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
plot_weekly('Total Confirmed', 'black')

In [ ]:
plot_weekly('New Confirmed', 'black')

In [ ]:
plot_weekly('Total Deceased', 'crimson')

In [ ]:
plot_weekly('New Deceased', 'crimson')

In [ ]:
plot_weekly('Total Recovered', 'green')

In [ ]:
plot_weekly('New Recovered', 'green')

# Month wise

In [ ]:
month_wise = daily.groupby('Month')['Daily Confirmed', 'Total Confirmed', 'Daily Recovered', 
                                    'Total Recovered', 'Daily Deceased', 'Total Deceased'].sum().reset_index()
month_wise.columns = ['Month', 'New Confirmed', 'Total Confirmed', 'New Recovered', 
                      'Total Recovered', 'New Deceased', 'Total Deceased']
month_wise.head()

In [ ]:
def plot_monthly(col, hue):
    fig = px.bar(month_wise, x="Month", y=col, title=col, 
                 width=600, color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
plot_monthly('Total Confirmed', 'black')

In [ ]:
plot_monthly('New Confirmed', 'black')

In [ ]:
plot_monthly('Total Deceased', 'crimson')

In [ ]:
plot_monthly('New Deceased', 'crimson')

In [ ]:
plot_monthly('Total Recovered', 'Green')

In [ ]:
plot_monthly('New Recovered', 'green')